In [ ]:
import os
import torch
import torchvision.models as models

In [ ]:
'''
TODO : 
- Make mobilenetv2 architecture
  - Depthwise conv
    - Inverted Residual Block
  - Pointwise conv
  - BatchNorm
  


'''

### Model architecute print

In [ ]:
model = models.mobilenet_v2(weights=models.MobileNet_V2_Weights.IMAGENET1K_V1)
print(model)

MobileNetV2(
  (features): Sequential(
    (0): Conv2dNormActivation(
      (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU6(inplace=True)
    )
    (1): InvertedResidual(
      (conv): Sequential(
        (0): Conv2dNormActivation(
          (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
          (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (2): ReLU6(inplace=True)
        )
        (1): Conv2d(32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
    )
    (2): InvertedResidual(
      (conv): Sequential(
        (0): Conv2dNormActivation(
          (0): Conv2d(16, 96, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (1): BatchNorm2d(96, eps=

: 

In [12]:
from torchsummary import summary

model = models.mobilenet_v2(weights=models.MobileNet_V2_Weights.IMAGENET1K_V1)
summary(model, (3, 224, 224))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 32, 112, 112]             864
       BatchNorm2d-2         [-1, 32, 112, 112]              64
             ReLU6-3         [-1, 32, 112, 112]               0
            Conv2d-4         [-1, 32, 112, 112]             288
       BatchNorm2d-5         [-1, 32, 112, 112]              64
             ReLU6-6         [-1, 32, 112, 112]               0
            Conv2d-7         [-1, 16, 112, 112]             512
       BatchNorm2d-8         [-1, 16, 112, 112]              32
  InvertedResidual-9         [-1, 16, 112, 112]               0
           Conv2d-10         [-1, 96, 112, 112]           1,536
      BatchNorm2d-11         [-1, 96, 112, 112]             192
            ReLU6-12         [-1, 96, 112, 112]               0
           Conv2d-13           [-1, 96, 56, 56]             864
      BatchNorm2d-14           [-1, 96,

In [9]:
import torch
from torchvision.models import mobilenet_v2
from torch.fx import symbolic_trace, Node

model = mobilenet_v2(weights=None).eval()

gm = symbolic_trace(model)

print("===== FX GRAPH =====")
for node in gm.graph.nodes:
    print(f"{node.op} : {node.name} : {node.target} : {node.args}")

===== FX GRAPH =====
placeholder : x : x : ()
call_module : features_0_0 : features.0.0 : (x,)
call_module : features_0_1 : features.0.1 : (features_0_0,)
call_module : features_0_2 : features.0.2 : (features_0_1,)
call_module : features_1_conv_0_0 : features.1.conv.0.0 : (features_0_2,)
call_module : features_1_conv_0_1 : features.1.conv.0.1 : (features_1_conv_0_0,)
call_module : features_1_conv_0_2 : features.1.conv.0.2 : (features_1_conv_0_1,)
call_module : features_1_conv_1 : features.1.conv.1 : (features_1_conv_0_2,)
call_module : features_1_conv_2 : features.1.conv.2 : (features_1_conv_1,)
call_module : features_2_conv_0_0 : features.2.conv.0.0 : (features_1_conv_2,)
call_module : features_2_conv_0_1 : features.2.conv.0.1 : (features_2_conv_0_0,)
call_module : features_2_conv_0_2 : features.2.conv.0.2 : (features_2_conv_0_1,)
call_module : features_2_conv_1_0 : features.2.conv.1.0 : (features_2_conv_0_2,)
call_module : features_2_conv_1_1 : features.2.conv.1.1 : (features_2_conv_

### Export Model as ONNX && Viz. as Netron (optimized graph)

In [ ]:
def export_mobilenet_v2_onnx(
    onnx_path: str = "mobilenet_v2.onnx",
    # opset: int = 17,
):
    # 1. 모델 준비
    model = models.mobilenet_v2(weights=None).eval()

    # 2. 더미 입력
    dummy = torch.randn(1, 3, 224, 224)

    # 3. ONNX export
    torch.onnx.export(
        model,
        dummy,
        onnx_path,
        input_names=["input"],
        output_names=["output"],
        # opset_version=opset,
        do_constant_folding=True,
        dynamic_axes={
            "input": {0: "batch_size"},
            "output": {0: "batch_size"},
        },
    )

    print(f"Exported to: {onnx_path}")


export_mobilenet_v2_onnx() # netron mobilenet_v2.onnx